In [21]:
import numpy
import typing
import sys
sys.path.append('..')

In [22]:
def convolve1D_circular(signal: numpy.ndarray, kernel: numpy.ndarray) -> numpy.ndarray:

    out_shape = signal.shape[0] + kernel.shape[0] - 1
    out = numpy.zeros(shape=out_shape)

    for i in range(0, out_shape):
        for k in range(0, kernel.shape[0]):
            s_idx = (i - (kernel.shape[0]//2 - 1) + k) % signal.shape[0]
            k_idx = kernel.shape[0]-1-k
            out[i] += signal[s_idx] * kernel[k_idx]
        
    return out

def convolve1D_zero_padding(signal: numpy.ndarray, kernel: numpy.ndarray) -> numpy.ndarray:

    out_shape = signal.shape[0] + kernel.shape[0] - 1
    out = numpy.zeros(shape=out_shape)

    for i in range(0, out_shape):
        for k in range(0, kernel.shape[0]):
            s_idx = i - (kernel.shape[0]//2 - 1) + k
            if (0 <= s_idx) and (s_idx < signal.shape[0]):
                k_idx = kernel.shape[0]-1-k
                out[i] += signal[s_idx] * kernel[k_idx]
        
    return out

def convolve1D_padding_value(signal: numpy.ndarray, kernel: numpy.ndarray, padding_value: float) -> numpy.ndarray:

    out_shape = signal.shape[0] + kernel.shape[0] - 1
    out = numpy.zeros(shape=out_shape)

    for i in range(0, out_shape):
        for k in range(0, kernel.shape[0]):
            s_idx = i - (kernel.shape[0]//2 - 1) + k
            if (0 <= s_idx) and (s_idx < signal.shape[0]):
                k_idx = kernel.shape[0]-1-k
                out[i] += signal[s_idx] * kernel[k_idx]
            else:
                k_idx = kernel.shape[0]-1-k
                out[i] += padding_value * kernel[k_idx]
        
    return out



def convolve1D_zero_padding_v2(signal: numpy.ndarray, kernel: numpy.ndarray) -> numpy.ndarray:

    out_shape = signal.shape[0] + kernel.shape[0] - 1
    out = numpy.zeros(shape=out_shape)

    for i in range(0, out_shape):
        for k in range(0, kernel.shape[0]):
            s_idx = i - (kernel.shape[0]//2 - 1) + k
            if (0 <= s_idx) and (s_idx < signal.shape[0]):
                k_idx = kernel.shape[0]-1-k
                out[i] += signal[s_idx] * kernel[k_idx]
        
    return out






def convolve1D_full(signal: numpy.ndarray, kernel: numpy.ndarray) -> numpy.ndarray:
    
    out_shape = signal.shape[0] + kernel.shape[0] - 1
    
    to_convolve = numpy.zeros(shape=signal.shape[0] + 2*(kernel.shape[0]-1))
    to_convolve[kernel.shape[0]-1:-(kernel.shape[0]-1)] = signal
    
    out = numpy.zeros(shape=out_shape)
    reversed_kernel = kernel[::-1]
    
    for i in range(0, out_shape):
        for k in range(0, kernel.shape[0]):
            out[i] += to_convolve[i+k]*reversed_kernel[k]
            
    return out

    
def convolve2D_full(signal: numpy.ndarray, kernel: numpy.ndarray) -> numpy.ndarray:
    
    out_shape = [
        signal.shape[0] + kernel.shape[0] - 1,
        signal.shape[1] + kernel.shape[1] - 1
    ]
    
    to_convolve = numpy.zeros(
        shape = [
            signal.shape[0] + 2*(kernel.shape[0]-1),
            signal.shape[1] + 2*(kernel.shape[1]-1)
        ]
    )
    
    to_convolve[
        kernel.shape[0]-1:-(kernel.shape[0]-1),
        kernel.shape[1]-1:-(kernel.shape[1]-1)
    ] = signal
    
    out = numpy.zeros(shape=out_shape)
    reversed_kernel = kernel[::-1, ::-1]
    
    for i in range(0, out_shape[0]):
        for j in range(0, out_shape[1]):
            for k1 in range(0, kernel.shape[0]):
                for k2 in range(0, kernel.shape[1]):
                    out[i, j] += to_convolve[i+k1, j+k2]*reversed_kernel[k1, k2]
            
    return out

def gaussian1D(value: float, mu: float, sigma: float) -> typing.Union[float, numpy.ndarray]:
    out = numpy.exp( - (value - mu)**2 / (2*sigma**2) )
    # out /= ( 2*numpy.pi) ** (1/2)
    out /= numpy.sqrt(2*numpy.pi)
    out /= sigma
    return out
    
# @classmethod
# def gaussian2D(cls, x: float, y:float, mu_x: float, mu_y: float, sigma_x: float, sigma_y: float) -> float :
#     out_x = numpy.exp( - (x - mu_x)**2 / sigma_x**2 )
#     out_x /= ( 2*numpy.pi) ** (1/2) * sigma_x
#     out_y = numpy.exp( - (y - mu_y)**2 / sigma_y**2 )
#     out_y /= ( 2*numpy.pi) ** (1/2) * sigma_y
#     return out_x * out_y


def gaussian2D(x: float, y: float, mu_x: float, mu_y: float, sigma_x: float, sigma_y: float) -> typing.Union[float, numpy.ndarray]:
    gaussian_x = gaussian1D(x, mu_x, sigma_x)
    gaussian_y = gaussian1D(y, mu_y, sigma_y)
    return gaussian_x*gaussian_y
    

def gaussianND(xi: numpy.ndarray, mu: numpy.ndarray, sigma: numpy.ndarray) -> typing.Union[float, numpy.ndarray]:
    dim = len(xi)
    gauss1D =  [ gaussian1D(xi[i], mu[i], sigma[i]) for i in range(0, dim) ]
    out = numpy.multiply(*gauss1D)
    return out

In [23]:
size = numpy.array([3, 3])
dim = 2
params = [ numpy.arange(-size[i]//2+1, size[i]//2+1) for i in range(0, size.shape[0]) ]
xi = numpy.meshgrid(*params)
mu, sigma = 0.0, 1.0
gaussian2D(*xi, mu, mu, sigma, sigma)

array([[0.05854983, 0.09653235, 0.05854983],
       [0.09653235, 0.15915494, 0.09653235],
       [0.05854983, 0.09653235, 0.05854983]])

In [24]:
size = numpy.array([3, 3, 3])
params = [ numpy.arange(-size[i]//2+1, size[i]//2+1) for i in range(0, size.shape[0]) ]
xi = numpy.meshgrid(*params)
mu = numpy.array([0.0, 0.0, 0.0])
sigma = numpy.array([1.0, 1.0, 1.0])
gaussianND(xi, mu, sigma)

array([[[0.05854983, 0.05854983, 0.05854983],
        [0.09653235, 0.09653235, 0.09653235],
        [0.05854983, 0.05854983, 0.05854983]],

       [[0.09653235, 0.09653235, 0.09653235],
        [0.15915494, 0.15915494, 0.15915494],
        [0.09653235, 0.09653235, 0.09653235]],

       [[0.05854983, 0.05854983, 0.05854983],
        [0.09653235, 0.09653235, 0.09653235],
        [0.05854983, 0.05854983, 0.05854983]]])

In [25]:
import abc
import enum

import numpy

class BoundaryCondition(enum.Enum):
    PERIODIC = 0
    CIRCULAR = 1
    ZEROS = 2

class ConvolveType(enum.Enum):
    FULL = 0
    SAME = 0


# def convolve_same(x: numpy.ndarray, kernel: numpy.ndarray, )


class Filter(abc.ABC):
        
    @abc.abstractmethod
    def __call__(self, x: numpy.ndarray) -> numpy.ndarray:
        pass


class GaussianFilter1D(Filter):
    
    def __init__(self, size: int, mu: float, sigma: float) -> None:
        x = numpy.arange(-size//2 + 1, size//2 + 1)
        self.kernel = gaussian1D(x, mu, sigma)
        self.__mu = mu
        self.__sigma = sigma

    @property
    def mu(self) -> float:
        return self.__mu
    
    @mu.setter
    def mu(self, value) -> float:
        raise AssertionError('You cant set mu !')
    
    @property
    def sigma(self) -> float:
        return self.__sigma
    
    @sigma.setter
    def sigma(self, value) -> float:
        raise AssertionError('You cant set sigma !')
        
    def __call__(self, x) -> numpy.array:
        return convolve1D_full(x, self.kernel)
        


# class SeparableGaussianFilterND:
    
#     def __init__(self, size: numpy.ndarray, sigmas: numpy.ndarray) -> None:
#         dim = size.shape[0]
#         params = numpy.array([ numpy.arange(-size[i]//2+1, size[i]//2+1) for i in range(0, dim) ])
#         xi = numpy.meshgrid(*params)
#         self.filters = [ Functional.gaussian1D(x, sigma) for x, sigma in zip(xi, sigmas) ]
        
#     def __call__(self, signal: float) -> numpy.array:
#         out = numpy.copy(signal)
#         for filter in self.filters:
#             out = filter(out)
#         return out
     

# class GaussianFilter2D(Filter):
    
#     def __init__(self, size: numpy.ndarray, mu: numpy.ndarray, sigma: numpy.ndarray) -> None:
#         dim = size.shape[0]
#         params = [ numpy.arange(-size[i]//2 + 1, size[i]//2 + 1) for i in range(0, dim) ]
#         xi = numpy.meshgrid(*params)
#         self.kernel = gaussian2D(*xi, *mu, *sigma)

#     def __call__(self, x: numpy.ndarray) -> numpy.ndarray:
#         return convolve2D_full(x, self.kernel)

# class GaussianFilterND(Filter):
    
#     def __init__(self, size: numpy.ndarray, mu: numpy.ndarray, sigma: numpy.ndarray) -> None:
#         dim = size.shape[0]
#         params = [ numpy.arange(-size[i]//2 + 1, size[i]//2 + 1) for i in range(0, dim) ]
#         xi = numpy.meshgrid(*params)
#         self.kernel = gaussianND(xi, mu, sigma)


# class SequentialFilter:

#     def __init__(self, filters: list[filter]) -> None:
    

In [26]:
# #kernel = GaussianFilter1D(size=3, sigma=1.0)
# gaussian2D = GaussianFilterND(
#     size = numpy.array([3, 3]),
#     mu = numpy.array([0.0, 0.0]),
#     sigma = numpy.array([1.0, 1.0])
# )

# # nd = SeparableGaussianFilterND(
# #     size = numpy.array([3, 3]),
# #     sigmas = numpy.array([ 1.0, 1.0 ])
# # )

# print(gaussian2D.kernel)

# #print(kernel.array)
# #print(kernel(numpy.ones(4)))
# #print(grid)

# # print(numpy.reshape(numpy.arange(0, 16), (4, 4))[::-1, ::-1])

In [27]:
# import lasp.filters.linear
# kernel = lasp.filters.linear.gaussian_filter(size=3, sigma=1.0)

In [28]:
ones = numpy.ones((5, 5))
ones

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [29]:
# convolve2D_full(ones, kernel)

In [30]:
# import scipy.signal
# scipy.signal.convolve2d(ones, kernel, mode='full')

In [31]:
import numpy
import scipy.signal

def convolve1D_boundary_zero(
    x: numpy.ndarray, 
    y: numpy.ndarray
) -> numpy.ndarray:
    
    N, M = x.shape[0], y.shape[0]
    w = numpy.zeros(shape=N+M-1)
    
    x_pad = numpy.zeros(shape=N+2*(M-1))
    x_pad[M-1:-(M-1)] = x

    for n in range(0, N+M-1):
        for k in range(0, M):
            w[n] += x_pad[n+k]*y[M-1-k]
            
    return w


def convolve1D_boundary_zero(
    x: numpy.ndarray, 
    y: numpy.ndarray
) -> numpy.ndarray:
    
    N, M = x.shape[0], y.shape[0]
    w = numpy.zeros(shape=N+M-1)

    for n in range(0, N+M-1):
        
        x_start = max(0, n-(M-1))
        x_end = min(N, n+1)
        y_start = max(0, M-1-n)
        
        for k in range(0, x_end-x_start):
            w[n] += x[x_start+k] * y[M-1-(y_start+k)]
    
    return w

In [248]:
import numpy
import scipy.signal

# def convolve1D_boundary_zero(
#     x: numpy.ndarray, 
#     y: numpy.ndarray
# ) -> numpy.ndarray:
    
#     N, M = x.shape[0], y.shape[0]
#     w = numpy.zeros(shape=N+M-1)

#     for n in range(0, N+M-1):
        
#         x_start = max(0, n-(M-1))
#         x_end = min(N, n+1)
#         y_start = max(0, M-1-n)
#         # y_end = y_start+(x_end-x_start)
        
#         for k in range(0, x_end-x_start):
#             w[n] += x[x_start+k] * y[M-1-(y_start+k)]
        
#         # for k in range(0, x_end-x_start):
#         #     w[n] += x[x_start+k] * y[k]
#         #     print(x[x_start+k], y[k])
#         # # print(n, x_start, x_end)
#         # nb_value=x_end-x_start
#         # print(x_start, x_end)
#         # print(y[::-1])
#         print(x[x_start:x_end], [ y[::-1][M-(x_end-x_start)+k] for k in range(0, x_end-x_start)])
#         print(x[x_start:x_end], y[::-1][y_start:y_start+(x_end-x_start)], y_start, M-1-y_start)
#         print()
#         # y_start = min(n, M-1)
#         # for k in range(0, x_end-x_start):
#         #     w[n] += x[x_start+k]*y[y_start-k]
            
#     return w




In [249]:
# x = numpy.random.randint(low=0, high=10, size=4)
x = numpy.arange(0, 4)
y = numpy.arange(0, 3)
# gaussian = GaussianFilter1D(size=3, mu=0, sigma=1.0)
# y = gaussian.kernel
x, y, y[::-1]

(array([0, 1, 2, 3]), array([0, 1, 2]), array([2, 1, 0]))

In [252]:
convolve1D_boundary_zero(y, x), convolve1D_boundary_zero(x, y)

(array([0., 0., 1., 4., 7., 6.]), array([0., 0., 1., 4., 7., 6.]))

In [255]:
scipy.signal.convolve(y, x, mode='valid')

array([1, 4])

## Closest of C / C++ / Pseudo Code

In [8]:
import numpy
import scipy.signal

In [29]:
def pythonic_convolve_zeropad(
    x: numpy.ndarray, 
    y: numpy.ndarray
) -> numpy.ndarray:
    N, M = x.shape[0], y.shape[0]
    padded = numpy.pad(x, pad_width= (M-1, M-1), mode='constant', constant_values=0)
    return scipy.signal.convolve(padded, y, mode='valid')

def convolve1D_boundary_zero(
    x: numpy.ndarray, 
    y: numpy.ndarray
) -> numpy.ndarray:
    
    N, M = x.shape[0], y.shape[0]
    w = numpy.zeros(shape=N+M-1)

    for n in range(0, N+M-1):
        
        x_start = max(0, n-(M-1))
        x_end = min(N, n+1)
        y_start = max(0, M-1-n)
        
        for k in range(0, x_end-x_start):
            w[n] += x[x_start+k] * y[M-1-(y_start+k)]
    
    return w

In [30]:
# x = numpy.random.randint(low=0, high=10, size=4)
x = numpy.arange(0, 4, dtype=numpy.float32)
y = numpy.arange(0, 3, dtype=numpy.float32)
# gaussian = GaussianFilter1D(size=3, mu=0, sigma=1.0)
# y = gaussian.kernel
x, y



(array([0., 1., 2., 3.], dtype=float32), array([0., 1., 2.], dtype=float32))

In [31]:
convolve1D_boundary_zero(y, x), convolve1D_boundary_zero(x, y), scipy.signal.convolve(x, y, mode='full')

(array([0., 0., 1., 4., 7., 6.]),
 array([0., 0., 1., 4., 7., 6.]),
 array([0., 0., 1., 4., 7., 6.], dtype=float32))

In [32]:
pythonic_convolve_zeropad(y, x), pythonic_convolve_zeropad(x, y)

(array([0., 0., 1., 4., 7., 6.], dtype=float32),
 array([0., 0., 1., 4., 7., 6.], dtype=float32))

In [19]:
# def convolve1D_boundary_circular(
#     x: numpy.ndarray, 
#     y: numpy.ndarray
# ) -> numpy.ndarray:
    
#     N, M = x.shape[0], y.shape[0]
    
#     w = numpy.zeros( shape = N+M-1 )
#     for n in range( 0, N+M-1 ):
#         for k in range( 0, M ):
#             w[n] += x[ (n-k)%N ] * y[ k ]
#         #     print(x[(n-k)%N], y[ k ])
#         # print()
    
#     return w

def convolve1D_boundary_circular(
    x: numpy.ndarray, 
    y: numpy.ndarray
) -> numpy.ndarray:
    
    N, M = x.shape[0], y.shape[0]
    
    w = numpy.zeros( shape = N+M-1 )
    for n in range( 0, N+M-1 ):
        for k in range( 0, M ):
            w[n] += x[ (n-k)%N ] * y[ k ]
    
    return w

convolve1D_boundary_circular(x, y), convolve1D_boundary_circular(y, x) 

(array([7., 6., 1., 4., 7., 6.]), array([4., 7., 7., 4., 7., 7.]))

In [20]:
def pythonic_convolve_circular(
    x: numpy.ndarray, 
    y: numpy.ndarray
) -> numpy.ndarray:
    N, M = x.shape[0], y.shape[0]
    padded = numpy.pad(x, pad_width= (M-1, M-1), mode='wrap')
    return scipy.signal.convolve(padded, y, mode='valid')

pythonic_convolve_circular(x, y), pythonic_convolve_circular(y, x)

(array([7., 6., 1., 4., 7., 6.], dtype=float32),
 array([4., 7., 7., 4., 7., 7.], dtype=float32))

In [6]:
# rule = "2x²-4x+1"

# l = rule.split("²")

# char = ""
# if len(l) == 2:
#     char += (l[0] + "**2" + l[1])
# else:
#     char = l[0]
    
# l = char.split("x")
# l.reverse()
# char = l[0]
# d = 1
# for c in l[1:]:
#     char = (c + "*x") + char
    
# f = eval('lambda x:'+char)
# f(4)